In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.font_manager as fm
import re

# 맑은 고딕 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False  # 음수 기호가 깨지는 것을 방지

# 데이터 자치구별로 정리하기

In [4]:
# 자동차 등록 현황

df = pd.read_excel("자동차등록현황보고_자동차등록대수현황 시도별 (201101 ~ 202411).xlsx",
                  skiprows = 4, usecols = [0, 1, 2,22])
df

C:\Users\User\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,월(Monthly),시도명,시군구,총계.3
0,월(Monthly),시도명,시군구,계
1,2017-01,서울,계,"3,091,903"
2,2017-01,서울,강남구,"243,194"
3,2017-01,서울,강동구,"136,915"
4,2017-01,서울,강북구,"76,983"
...,...,...,...,...
25519,2024-11,전북,전주시 덕진구,"183,121"
25520,2024-11,전북,전주시 완산구,"161,746"
25521,2024-11,전북,정읍시,"61,367"
25522,2024-11,전북,진안군,"15,665"


In [6]:
df.columns = df.iloc[0]

# 0번째 행 삭제
df = df[2:].reset_index(drop=True)

# 서울로 지역 한정
df = df[df['시도명']=='서울']

# 연도와 월 분리
df[['년', '월']] = df['월(Monthly)'].str.split('-', expand=True)
df.drop(columns = ['월(Monthly)', '시도명'], inplace = True)

# 11월로 시기 고정
df = df[df['월'] == '11']

# 시별 총계값은 삭제
df = df[df['시군구'] != '계']

# 칼럼 정리
df.drop(columns = ['월'], inplace = True)
df.columns = ['시군구', '자동차등록수(대)', '년']

df

,시군구,자동차등록수(대),년
2682,강남구,"239,036",2017
2683,강동구,"137,016",2017
2684,강북구,"77,945",2017
2685,강서구,"201,590",2017
2686,관악구,"122,287",2017
...,...,...,...
25270,용산구,"73,224",2024
25271,은평구,"134,044",2024
25272,종로구,"50,150",2024
25273,중구,"52,829",2024


In [7]:
df.to_csv('수정_서울시_자동차등록수(11월기준)(2017-2024).csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 2682 to 25274
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   시군구        200 non-null    object
 1   자동차등록수(대)  200 non-null    object
 2   년          200 non-null    object
dtypes: object(3)
memory usage: 6.2+ KB


In [11]:
# 전기차 데이터 (21년까지)

df= pd.read_excel("15~21년 행정동별 연료별 자동차 등록현황.xlsx",
                 header = 1, sheet_name= '시군구별 행정동별 용도별 차종별 연료별 건수')

df

,연월,사용본거지시군구,시군구코드,행정동사용본거지,행정동사용본거지코드,용도,차종종별,연료,건수
0,201512,서울특별시 종로구,11110,NaN,58000,자가용,화물,경유,1
1,201512,서울특별시 종로구,11110,NaN,60000,자가용,승용,휘발유,2
2,201512,서울특별시 종로구,11110,NaN,60000,자가용,화물,휘발유,1
3,201512,서울특별시 종로구,11110,NaN,61500,자가용,화물,휘발유,2
4,201512,서울특별시 종로구,11110,NaN,62000,자가용,화물,경유,1
...,...,...,...,...,...,...,...,...,...
101556,202112,서울특별시 강동구,11740,서울특별시 강동구 둔촌1동,1174069000,영업용,승합,엘피지,76
101557,202112,서울특별시 강동구,11740,서울특별시 강동구 둔촌1동,1174069000,영업용,화물,전기,1
101558,202112,서울특별시 강동구,11740,서울특별시 강동구 둔촌2동,1174070000,자가용,승용,경유,1894
101559,202112,서울특별시 강동구,11740,서울특별시 강동구 둔촌2동,1174070000,자가용,화물,기타연료,6


In [12]:
# 주소 분리 및 필요 없는 컬럼 삭제
df[['시', '시군구']] = df['사용본거지시군구'].str.split(' ', expand=True)
df.drop(columns = ['사용본거지시군구', '시', '시군구코드', '용도', '차종종별'], inplace = True)

# 연도와 월 분리
df['년'] = df['연월'].astype(str).str[:4]  # 앞에서 4글자 추출
df['월'] = df['연월'].astype(str).str[4:6]

df.drop(columns = ['연월'], inplace = True)

# 전기차 한정
df = df[df['연료'] == '전기']

# 시군구별 년도별 전기차 등록 수 합하기
a = df.groupby(['시군구', '년'])['건수'].sum().reset_index()

# 컬럼명 정리
a.columns = ['시군구', '년', '전기차(대)']
a

,시군구,년,전기차(대)
0,강남구,2015,155
1,강남구,2016,218
2,강남구,2017,2385
3,강남구,2018,4262
4,강남구,2019,5756
...,...,...,...
170,중랑구,2017,40
171,중랑구,2018,128
172,중랑구,2019,212
173,중랑구,2020,412


In [13]:
# 전기차 데이터 (22년)

df_22 = pd.read_excel('22.12월 자치구별연료별차종별용도별.xls', skiprows = 7)
df_22

,시군구별,Unnamed: 1,Unnamed: 2,연료별,Unnamed: 4,Unnamed: 5,용도별,승 용,승 합,화 물,Unnamed: 10,Unnamed: 11,특 수,Unnamed: 13,Unnamed: 14,계
0,합 계,NaN,NaN,계,NaN,NaN,계,2763730,96085,322706,NaN,NaN,10830,NaN,NaN,3193351
1,종로구,NaN,NaN,CNG,NaN,NaN,비사업용,6,4,19,NaN,NaN,0,NaN,NaN,29
2,NaN,NaN,NaN,CNG,NaN,NaN,사업용,0,84,0,NaN,NaN,0,NaN,NaN,84
3,NaN,NaN,NaN,경유,NaN,NaN,비사업용,9852,2944,3564,NaN,NaN,142,NaN,NaN,16502
4,NaN,NaN,NaN,경유,NaN,NaN,사업용,147,157,761,NaN,NaN,74,NaN,NaN,1139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,NaN,NaN,NaN,휘발유,NaN,NaN,비사업용,29332,18,106,NaN,NaN,0,NaN,NaN,29456
554,NaN,NaN,NaN,휘발유,NaN,NaN,사업용,81,0,0,NaN,NaN,0,NaN,NaN,81
555,NaN,NaN,NaN,휘발유(무연),NaN,NaN,비사업용,48462,16,29,NaN,NaN,0,NaN,NaN,48507
556,NaN,NaN,NaN,휘발유(무연),NaN,NaN,사업용,269,6,0,NaN,NaN,0,NaN,NaN,275


In [14]:
# 시군구명 바로 위 값으로 na 대체하기
df_22['시군구별'] = df_22['시군구별'].fillna(method='ffill')

# 전기차만 저장
df_22 = df_22[df_22['연료별'] == '전기']


df_22 = df_22.groupby(['시군구별'])['계'].sum().reset_index()
df_22['년'] = 2022

df_22.columns = ['시군구', '전기차(대)', '년'] 
df_22

,시군구,전기차(대),년
0,강남구,13924,2022
1,강동구,2250,2022
2,강북구,880,2022
3,강서구,2857,2022
4,관악구,1343,2022
5,광진구,1065,2022
6,구로구,5138,2022
7,금천구,1024,2022
8,노원구,1574,2022
9,도봉구,1106,2022


In [15]:
# 전기차 데이터 (23년)

df_23 = pd.read_excel('서울시 자치구별 연료별 차종별 용도별 등록현황(23년12월).xls', skiprows = 7)
df_23

,시군구별,Unnamed: 1,Unnamed: 2,연료별,Unnamed: 4,Unnamed: 5,용도별,승 용,승 합,화 물,Unnamed: 10,Unnamed: 11,특 수,Unnamed: 13,Unnamed: 14,계
0,합 계,NaN,NaN,계,NaN,NaN,계,2776116,91067,312590,NaN,NaN,11389,NaN,NaN,3191162
1,종로구,NaN,NaN,CNG,NaN,NaN,비사업용,6,4,16,NaN,NaN,0,NaN,NaN,26
2,NaN,NaN,NaN,CNG,NaN,NaN,사업용,0,80,0,NaN,NaN,0,NaN,NaN,80
3,NaN,NaN,NaN,경유,NaN,NaN,비사업용,9475,2766,3499,NaN,NaN,142,NaN,NaN,15882
4,NaN,NaN,NaN,경유,NaN,NaN,사업용,53,168,1212,NaN,NaN,94,NaN,NaN,1527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,NaN,NaN,NaN,휘발유,NaN,NaN,비사업용,28381,17,111,NaN,NaN,0,NaN,NaN,28509
554,NaN,NaN,NaN,휘발유,NaN,NaN,사업용,79,0,0,NaN,NaN,0,NaN,NaN,79
555,NaN,NaN,NaN,휘발유(무연),NaN,NaN,비사업용,49678,18,30,NaN,NaN,0,NaN,NaN,49726
556,NaN,NaN,NaN,휘발유(무연),NaN,NaN,사업용,282,6,0,NaN,NaN,0,NaN,NaN,288


In [16]:
# 시군구명 바로 위 값으로 na 대체하기
df_23['시군구별'] = df_23['시군구별'].fillna(method='ffill')

# 전기차만 저장
df_23 = df_23[df_23['연료별'] == '전기']


df_23 = df_23.groupby(['시군구별'])['계'].sum().reset_index()
df_23['년'] = 2023

df_23.columns = ['시군구', '전기차(대)', '년'] 
df_23

,시군구,전기차(대),년
0,강남구,14215,2023
1,강동구,2965,2023
2,강북구,1189,2023
3,강서구,3683,2023
4,관악구,1815,2023
5,광진구,1381,2023
6,구로구,5615,2023
7,금천구,1355,2023
8,노원구,2172,2023
9,도봉구,1472,2023


In [19]:
# 전기차 데이터 (24년)

df_24 = pd.read_excel('서울시 자치구별 연료별 차종별 용도별 등록현황(24년11월).xls', skiprows = 7)
df_24

,시군구별,Unnamed: 1,Unnamed: 2,연료별,Unnamed: 4,Unnamed: 5,용도별,승 용,승 합,화 물,Unnamed: 10,Unnamed: 11,특 수,Unnamed: 13,Unnamed: 14,계
0,합 계,NaN,NaN,계,NaN,NaN,계,2774447,85922,305468,NaN,NaN,11590,NaN,NaN,3177427
1,종로구,NaN,NaN,CNG,NaN,NaN,비사업용,5,7,13,NaN,NaN,0,NaN,NaN,25
2,NaN,NaN,NaN,CNG,NaN,NaN,사업용,0,66,0,NaN,NaN,0,NaN,NaN,66
3,NaN,NaN,NaN,경유,NaN,NaN,비사업용,8999,2663,3336,NaN,NaN,142,NaN,NaN,15140
4,NaN,NaN,NaN,경유,NaN,NaN,사업용,0,149,1164,NaN,NaN,93,NaN,NaN,1406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,NaN,NaN,NaN,휘발유,NaN,NaN,비사업용,27913,16,111,NaN,NaN,0,NaN,NaN,28040
554,NaN,NaN,NaN,휘발유,NaN,NaN,사업용,57,0,0,NaN,NaN,0,NaN,NaN,57
555,NaN,NaN,NaN,휘발유(무연),NaN,NaN,비사업용,51999,19,33,NaN,NaN,0,NaN,NaN,52051
556,NaN,NaN,NaN,휘발유(무연),NaN,NaN,사업용,342,0,0,NaN,NaN,0,NaN,NaN,342


In [20]:
# 시군구명 바로 위 값으로 na 대체하기
df_24['시군구별'] = df_24['시군구별'].fillna(method='ffill')

# 전기차만 저장
df_24 = df_24[df_24['연료별'] == '전기']


df_24 = df_24.groupby(['시군구별'])['계'].sum().reset_index()
df_24['년'] = 2024

df_24.columns = ['시군구', '전기차(대)', '년'] 
df_24

,시군구,전기차(대),년
0,강남구,14045,2024
1,강동구,3667,2024
2,강북구,1411,2024
3,강서구,4162,2024
4,관악구,2148,2024
5,광진구,1654,2024
6,구로구,5639,2024
7,금천구,1599,2024
8,노원구,2682,2024
9,도봉구,1874,2024


In [21]:
# 전기차 데이터 ~ 24 병합
df = pd.concat([a,df_22, df_23, df_24], axis=0, ignore_index = True)

df.to_csv('서울시_전기차(15-24 12월기준).csv')
df

,시군구,년,전기차(대)
0,강남구,2015,155
1,강남구,2016,218
2,강남구,2017,2385
3,강남구,2018,4262
4,강남구,2019,5756
...,...,...,...
245,용산구,2024,1895
246,은평구,2024,2593
247,종로구,2024,1508
248,중구,2024,2406


In [23]:
# 자치단체 재정자주도

df = pd.read_excel('자치단체 재정자주도_20241220151906.xlsx', skiprows = 28, 
                  nrows = 8 )
df

C:\Users\User\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,전국평균,특·광역시,도,시,군,자치구
0,2017,74.9,74.6,49.1,67.2,64.1,48.1
1,2018,75.3,73.4,50.2,66.4,65.2,47.5
2,2019,74.2,71.8,48.9,64.8,65.3,46.1
3,2020,73.9,69.8,50.6,63.8,64.9,45.5
4,2021,70.8,67.4,46.5,60.9,61.2,44.8
5,2022,73.4,70.0,50.7,62.5,64.2,45.2
6,2023,74.1,70.3,51.0,62.1,65.0,45.0
7,2024,70.9,66.4,47.0,59.9,62.4,43.0


In [24]:
# 컬럼명 정리
df = df[['Unnamed: 0', '군']]
df.columns = ['년', '재정자주도(군)']

df.to_csv('전국 군_재정자주도(17-24).csv')
df

,년,재정자주도(군)
0,2017,64.1
1,2018,65.2
2,2019,65.3
3,2020,64.9
4,2021,61.2
5,2022,64.2
6,2023,65.0
7,2024,62.4


In [26]:
# 충전기 데이터
df = pd.read_csv("한국환경공단_전기차 충전소 위치 및 운영정보(충전소 ID 포함)2017-2024.csv",
                 encoding = 'cp949')
df

C:\Users\User\AppData\Local\Temp\ipykernel_40620\1328751955.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("한국환경공단_전기차 충전소 위치 및 운영정보(충전소 ID 포함)2017-2024.csv",


,설치년도,시도,군구,주소,충전소명,시설구분(대),시설구분(소),기종(대),기종(소),운영기관(대),운영기관(소),급속충전량,충전기타입,이용자제한,위도경도
0,2017,서울특별시,강서구,서울특별시 강서구 가로공원로 189,가로공원로 지하공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5372628,126.8383789"
1,2017,서울특별시,강서구,서울특별시 강서구 강서로5길 50,곰달래문화복지센터 공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5289561,126.8490887"
2,2017,서울특별시,강서구,서울특별시 강서구 까치산로4길 22,볏골공원 지하공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5419747,126.8444499"
3,2017,서울특별시,강서구,서울특별시 강서구 우장산로 114,화곡6-1 공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.5530457,126.8488169"
4,2017,서울특별시,관악구,서울특별시 관악구 관천로 98,신림동 제1공영주차장,주차시설,공영주차장,급속,급속(50kW),환경부,환경부(협회),급속(50kW),DC차데모+AC3상+DC콤보,이용가능,"37.4874046,126.9235075"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361158,2015,광주광역시,남구,광주광역시 남구 화산로 30,진월국제테니스장,공공시설,지자체시설,급속,DC차데모+AC3상+DC콤보,지자체,지자체기타,NaN,DC차데모+AC3상+DC콤보,이용자제한,NaN
361159,2016,전라남도,영광군,전라남도 영광군 군서면 백수로 1481,농업기술센터,공공시설,지자체시설,완속,AC완속,지자체,지자체기타,NaN,AC완속,이용자제한,NaN
361160,2016,전라남도,영광군,전라남도 영광군 영광읍 월현로 170,스포티움 수영장,공공시설,지자체시설,완속,AC완속,지자체,지자체기타,NaN,AC완속,이용자제한,NaN
361161,2011,전라남도,강진군,전라남도 강진군 강진읍 탐진로 111,강진군청,공공시설,관공서,완속,AC완속,지자체,지자체기타,NaN,AC완속,이용자제한,NaN


In [30]:
# 서울특별시 지역 한정
df = df[df['시도'] == '서울특별시']

# 피벗테이블: 완속, 급속 개수 구하기
pivot_df = df.pivot_table(index=['군구', '설치년도'], columns='기종(대)', aggfunc='size', fill_value=0)

pivot_df.to_csv('군구별완속급속충전기.csv')
pivot_df


기종(대)      급속   완속
군구  설치년도          
강남구 2013    0    2
    2017   34   91
    2018   20  103
    2019   16   94
    2020   19  148
...       ...  ...
중랑구 2020    8   70
    2021   20  234
    2022   50  858
    2023  104  370
    2024    7  255

[233 rows x 2 columns]

# 최종 DF 만들기 / train data
### (자치구별 데이터 만들어서 병합)

In [3]:
# 각 데이터 불러오기

charger = pd.read_csv("군구별완속급속충전기.csv")
culture = pd.read_csv("서울시 구별 교통문화지수(2017~2023).csv")
e_car = pd.read_csv("서울시_전기차(15-24 12월기준).csv")
car = pd.read_csv("수정_서울시_자동차등록수(11월기준)(2017-2024).csv")
indep = pd.read_csv("전국 군_재정자주도(17-24).csv")
work = pd.read_csv("사업체 수 & 전기차 수 관계 수정본.csv", encoding = 'cp949')
house = pd.read_csv("서울시 구별 인구_가구_주택수.csv")
people = pd.read_excel('(공표용)서울특별시+자치구별+장래인구추계(2020~2040) (1).xlsx', sheet_name = '1. 자치구별 총인구', header = 2, nrows = 26)


In [4]:
# 데이터 가공(컬럼명 통일)

# 충전기 데이터 가공하기
charger = charger.rename(columns = {'군구' : '자치구', '설치년도' : '년도'})
#24년 데이터 제외
charger = charger[charger['년도'] != 2024]
#자치구별 타입별 누적개수 구하기
charger['급속'] = charger.groupby('자치구')['급속'].cumsum()
charger['완속'] = charger.groupby('자치구')['완속'].cumsum()

charger.to_csv('급속완속충전기_누적대수2024삭제.csv')

# 전기차 데이터 가공하기
e_car.drop(columns = ['Unnamed: 0'], inplace = True)
e_car = e_car.rename(columns = {'시군구' : '자치구', '년' : '년도'})

# 자동차등록수 데이터 가공하기
car.drop(columns = ['Unnamed: 0'], inplace = True)
car = car.rename(columns = {'시군구' : '자치구', '년' : '년도'})

# 재정자주도 데이터 가공하기
indep.drop(columns = ['Unnamed: 0'], inplace = True)
indep = indep.rename(columns = {'년' : '년도'})

# 사업체 데이터 가공하기
work = work.rename(columns = {'연도' : '년도'})

# 가구 데이터 가공하기
house = house.rename(columns = {'연도' : '년도'})

# 인구 데이터 가공하기
people.drop(columns = ['구분', '남여별', '자치구코드'], inplace = True)
people = people[['자치구', 2024]]
people = people.melt(id_vars = ['자치구'], var_name = '년도', value_name= '총인구 (명)')
people['자치구'] = people['자치구'].str.replace(' ', '')


In [5]:
#데이터 년도, 자치구를 key로 merge하기

df = pd.merge(e_car, car, on = ['년도', '자치구'], how = 'right')
df = pd.merge(df, charger, on = ['년도', '자치구'], how = 'left')
df = pd.merge(df, indep, on = ['년도'], how = 'left')
df = pd.merge(df, culture, on = ['년도', '자치구'], how = 'left')
df= pd.merge(df, work, on = ['년도', '자치구'], how = 'left')

df.drop(columns = ['전기차(대)_y'], inplace = True)
df = df.rename(columns = {'전기차(대)_x' : '전기차(대)'})

df = pd.merge(df, house, on = ['년도', '자치구'], how = 'left')


# 2024 데이터 장래값으로 대체하기
df.loc[df['년도'] == 2024, '총인구 (명)'] = np.nan
df[df['년도'] == 2024]['총인구 (명)']

df = df.merge(
    people[['년도', '자치구', '총인구 (명)']],
    on=['년도', '자치구'],
    how='left',
    suffixes=('', '_new')
)

# 총인구 값 대치
df['총인구 (명)'] = df['총인구 (명)_new'].combine_first(df['총인구 (명)'])

# 불필요한 열 제거
df = df.drop(columns=['총인구 (명)_new'])
df


,자치구,년도,전기차(대),자동차등록수(대),급속,완속,재정자주도(군),교통문화지수,부동산업_사업체수,전문 과학 및 기술 서비스업_사업체수,...,숙박 및 음식점업_종사자수,교육 서비스업_종사자수,금융 및 보험업_사업체수,수도 하수 및 폐기물 처리 원료 재생업_종사자수,건설업_종사자수,전기 가스 증기 및 공기조절 공급업_종사자수,총인구 (명),가구-계 (가구),공동주택/가구수,나머지주택/가구수
0,강남구,2017,2385,"239,036",34.0,93.0,64.1,84.84,5686,8798,...,56982,29001,1951,856,55618,1052,522514.0,210728.0,0.632830,0.189472
1,강동구,2017,81,"137,016",20.0,39.0,64.1,82.59,1531,602,...,15466,11239,278,511,17302,94,423978.0,163351.0,0.482483,0.257886
2,강북구,2017,27,"77,945",7.0,25.0,64.1,78.60,789,275,...,10569,5729,253,326,2057,80,313698.0,129204.0,0.296740,0.434863
3,강서구,2017,132,"201,590",21.0,71.0,64.1,92.46,1909,989,...,20057,13832,298,639,13334,426,581675.0,234926.0,0.494500,0.306501
4,관악구,2017,45,"122,287",14.0,22.0,64.1,83.39,1402,586,...,17322,15100,180,171,9125,15,511222.0,242972.0,0.247131,0.263915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,용산구,2024,1895,"73,224",NaN,NaN,62.4,NaN,2063,1941,...,16474,7502,576,268,6066,29,215882.0,NaN,NaN,NaN
196,은평구,2024,2593,"134,044",NaN,NaN,62.4,NaN,2181,1377,...,11704,10754,238,272,6887,163,453471.0,NaN,NaN,NaN
197,종로구,2024,1508,"50,150",NaN,NaN,62.4,NaN,2596,2699,...,23007,13330,947,128,25523,484,141438.0,NaN,NaN,NaN
198,중구,2024,2406,"52,829",NaN,NaN,62.4,NaN,3046,3653,...,27591,9644,2920,395,7322,704,122700.0,NaN,NaN,NaN


In [6]:
print(df.info())

df.to_csv('final_data_수정.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 38 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   자치구                           200 non-null    object 
 1   년도                            200 non-null    object 
 2   전기차(대)                        200 non-null    int64  
 3   자동차등록수(대)                     200 non-null    object 
 4   급속                            175 non-null    float64
 5   완속                            175 non-null    float64
 6   재정자주도(군)                      200 non-null    float64
 7   교통문화지수                        175 non-null    float64
 8   부동산업_사업체수                     200 non-null    int64  
 9   전문 과학 및 기술 서비스업_사업체수          200 non-null    int64  
 10  부동산업_종사자수                     200 non-null    int64  
 11  정보통신업_사업체수                    200 non-null    int64  
 12  보건업 및 사회복지 서비스업_사업체수          200 non-null    int64  
 13  합계_사업

# 시계열 예측 데이터 불러오기 / test data
### (2025년, 2026년 데이터 병합하기)

In [9]:
f_df = pd.read_csv("forecast_results2.csv")
f_charger = pd.read_csv('급속완속예측.csv', encoding = 'cp949')
f_house = pd.read_csv("가구 수 추계값 (2024-2031).csv")
people = pd.read_excel('(공표용)서울특별시+자치구별+장래인구추계(2020~2040) (1).xlsx',
                    sheet_name = '1. 자치구별 총인구', 
                      header = 2, nrows = 26)
home = pd.read_csv('가구수비율 예측.csv')

In [10]:
f_df.drop(columns = ['급속', '완속'], inplace = True)

f_charger = f_charger.pivot(index = ['자치구', '년도'], 
                        columns = '변수', values = '예측값')

f_charger = f_charger.reset_index()
f_charger['급속'] = round(f_charger['급속'])
f_charger['완속'] = round(f_charger['완속'])

f_charger = f_charger[f_charger['년도']!= 2024]

f_house = pd.read_csv("가구 수 추계값 (2024-2031).csv")
f_house.drop(columns = 'Unnamed: 0',  inplace = True)

people.drop(columns = ['구분', '남여별', '자치구코드'], inplace = True)
people = people[['자치구', 2025, 2026]]
people = people.melt(id_vars = ['자치구'], var_name = '년도', value_name= '총인구 (명)')
people['자치구'] = people['자치구'].str.replace(' ', '')

home = home.pivot(index = ['자치구', '년도'], columns = '변수', values = '예측값')
home = home.reset_index()


In [11]:
f_df = pd.merge(f_df, f_charger, on = ['자치구', '년도'], how = 'left')
f_df = pd.merge(f_df, f_house, on = ['자치구', '년도'], how = 'left')
f_df = pd.merge(f_df, people, on = ['자치구', '년도'], how = 'left')
f_df = pd.merge(f_df, home, on = ['자치구', '년도'], how = 'left')
f_df

,자치구,년도,건설업_종사자수,교육 서비스업_사업체수,교육 서비스업_종사자수,교통문화지수,금융 및 보험업_사업체수,도매 및 소매업_종사자수,보건업 및 사회복지 서비스업_사업체수,보건업 및 사회복지 서비스업_종사자수,...,합계_사업체수,합계_종사자수,협회 및 단체 수리 및 기타 개인서비스업_사업체수,협회 및 단체 수리 및 기타 개인서비스업_종사자수,급속,완속,가구-계 (가구),총인구 (명),공동주택/가구수,나머지주택/가구수
0,강남구,2025,52954.000000,5378.000000,33342.000000,78.327143,4184.000000,151552.000000,3516.000000,48952.285714,...,108906.0,802164.000000,5970.0,22154.000000,453.0,6200.0,215485.0,507421,0.629118,0.193087
1,강남구,2026,52954.000000,5378.000000,33342.000000,78.327143,4184.000000,151552.000000,3516.000000,50164.571429,...,108906.0,802164.000000,5970.0,22154.000000,490.0,7272.0,216450.0,507638,0.629118,0.193087
2,강동구,2025,22610.000000,2175.000000,11812.000000,79.161429,374.000000,28556.000000,1488.857143,25434.142857,...,42408.0,161149.000000,3387.0,5996.857143,189.0,2185.0,186315.0,439125,0.500078,0.264301
3,강동구,2026,22610.000000,2262.000000,11812.000000,79.161429,374.000000,28556.000000,1520.714286,26446.285714,...,42408.0,161149.000000,3387.0,5887.714286,209.0,2427.0,187149.0,436842,0.500078,0.264301
4,강북구,2025,4044.000000,889.000000,6274.000000,78.097143,346.000000,12534.000000,871.000000,11574.000000,...,26250.0,80040.000000,2523.0,4155.000000,309.0,940.0,130246.0,278538,0.300914,0.438899
5,강북구,2026,4044.000000,889.000000,6274.000000,78.097143,346.000000,12534.000000,871.000000,11574.000000,...,26250.0,80040.000000,2523.0,4109.000000,397.0,1044.0,130829.0,276497,0.300914,0.438899
6,강서구,2025,21032.285714,2633.857143,15201.142857,81.492857,599.714286,47191.857143,1786.571429,26832.285714,...,59233.0,303541.428571,4468.0,10088.000000,506.0,4228.0,250633.0,553983,0.483558,0.308457
7,강서구,2026,21994.571429,2739.714286,15372.285714,81.492857,637.428571,48757.714286,1817.142857,27948.571429,...,59233.0,312184.857143,4468.0,10088.000000,604.0,4830.0,251755.0,554012,0.483558,0.308457
8,관악구,2025,10608.000000,1582.000000,17960.571429,78.078571,232.000000,17132.000000,1286.285714,16520.000000,...,37364.0,127406.000000,3392.0,6040.000000,238.0,1115.0,266727.0,480294,0.238148,0.253715
9,관악구,2026,10608.000000,1582.000000,18318.142857,78.078571,232.000000,17132.000000,1300.571429,17156.000000,...,37364.0,127406.000000,3392.0,6040.000000,242.0,1133.0,267921.0,475844,0.238148,0.253715


In [12]:
f_df.info()
df.to_csv('test.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   자치구                           50 non-null     object 
 1   년도                            50 non-null     object 
 2   건설업_종사자수                      50 non-null     float64
 3   교육 서비스업_사업체수                  50 non-null     float64
 4   교육 서비스업_종사자수                  50 non-null     float64
 5   교통문화지수                        50 non-null     float64
 6   금융 및 보험업_사업체수                 50 non-null     float64
 7   도매 및 소매업_종사자수                 50 non-null     float64
 8   보건업 및 사회복지 서비스업_사업체수          50 non-null     float64
 9   보건업 및 사회복지 서비스업_종사자수          50 non-null     float64
 10  부동산업_사업체수                     50 non-null     float64
 11  부동산업_종사자수                     50 non-null     float64
 12  사업시설 관리 사업 지원 및 임대 서비스업_사업체수  50 non-null     float64
 13  사업시설 관리